# Technical indicators
domain specific features that translate raw price and volumes into actionable signals

## Load data

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pandas_ta as ta

# data loading
tickers = ['AAPL', 'MSFT', 'TSLA', 'AMZN', 'NVDA', 'SPY']
start_date = '2018-01-01'
end_date = '2024-01-01'

data = yf.download(tickers, start=start_date, end=end_date)

prices = data['Close'].ffill()
volumes = data['Volume'].ffill()

# combining data for easier indicator calculation
df = pd.DataFrame({'Close': prices['MSFT'], 'Volume': volumes['MSFT']})

/tmp/ipykernel_1269/1968664777.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date)
[*********************100%***********************]  6 of 6 completed


## Trend-following indicators
they smooth price action to identify the underlying direction and speed of the trend
### Simple moving average (SMA)
average price over a specific number of periods n.
defines current trend. if price is above SMA, the short term trend is bullish if below its bearish

In [2]:
df['SMA_5'] = ta.sma(df['Close'], length=5)     # Very Short-Term
df['SMA_20'] = ta.sma(df['Close'], length=20)   # Short-Term/MONTHLY TREND
df['SMA_50'] = ta.sma(df['Close'], length=50)   # Medium-Term

In [5]:
df['SMA_20'].tail()

Date
2023-12-22    368.041556
2023-12-26    367.846962
2023-12-27    367.421799
2023-12-28    367.245923
2023-12-29    367.104532
Name: SMA_20, dtype: float64

### Exponential moving average (EMA)
similar to SMA, but give weight to recent prices. more responsive to new infomation
better at tracking current price movement, useful for setting entry/exit points

In [6]:
df['EMA_12'] = ta.ema(df['Close'], length=12)
df['EMA_26'] = ta.ema(df['Close'], length=26)
# 12 AND 26 COMMON FOR MACD CALCULATION

In [7]:
df['EMA_12'].tail(), df['EMA_12'].head()

(Date
 2023-12-22    367.036013
 2023-12-26    367.362161
 2023-12-27    367.548688
 2023-12-28    367.889943
 2023-12-29    368.293903
 Name: EMA_12, dtype: float64,
 Date
 2018-01-02   NaN
 2018-01-03   NaN
 2018-01-04   NaN
 2018-01-05   NaN
 2018-01-08   NaN
 Name: EMA_12, dtype: float64)

## Momentum indicators
measure speed and strenght of price changes used to identify potential reversals
### Moving average convergence divergence (MACD)
measures difference between fast EMA (12 DAY) and slow ema(26day) this forms the MACD line and a 9 day EMA line is used as signal line. when they cross it can be a trigger for buy or sell signals.
 MACD crosses above signal line -> buy. when MACD cross below signal -> sell.
captures changes in momentum

In [8]:
macd_data = ta.macd(df['Close'], fast=12, slow=26, signal=9)
df = df.join(macd_data)

In [9]:
df[macd_data.columns].tail()

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
Date,,,
2023-12-22,2.800176,-1.032733,3.832908
2023-12-26,2.761869,-0.856831,3.618700
2023-12-27,2.654004,-0.771757,3.425761
2023-12-28,2.634358,-0.633122,3.267480
2023-12-29,2.648681,-0.495039,3.143721


### Relative strenght index (RSI)
a price oscillator that measures the magnitude of recent price gains against recent price losses normalized from 0 to 100
identifies overbought (>70) and oversold (<30) conditions suggesting a price correction or reversal is due

In [10]:
df['RSI_14'] = ta.rsi(df['Close'], length=14)

## Volatality and volume indicator
capture ampliture of price swings and the conviction behind those swings.
### Bollinger bands (BBANDS)
Consists of a middle band (20 day sma) and upper and lower bands that are typically 2 standard deviations away fro mthe middle band.
measures volatility and price extremes.
When bands narrow, low volatility may signal an impending breakout. Prices hitting the outer bands suggest short-term extreme highs or lows.

In [11]:
bbands_data = ta.bbands(df['Close'], length=20, std=2)
df = df.join(bbands_data)

df[bbands_data.columns].tail()

,BBL_20_2.0_2.0,BBM_20_2.0_2.0,BBU_20_2.0_2.0,BBB_20_2.0_2.0,BBP_20_2.0_2.0
Date,,,,,
2023-12-22,360.206767,368.041556,375.876345,4.257557,0.566089
2023-12-26,360.349661,367.846962,375.344263,4.076315,0.587299
2023-12-27,361.288197,367.421799,373.555400,3.338725,0.593973
2023-12-28,361.641206,367.245923,372.850639,3.052296,0.724893
2023-12-29,362.029889,367.104532,372.179175,2.764686,0.836098


### Average true range (ATR)
Measures market volatility by accounting for gaps and limit moves. The "True Range" is the largest of three differences: (High-Low, |High-Yesterday's Close|, |Low-Yesterday's Close|). ATR is the moving average of the True Range.
Primarily used to set stop-loss orders and measure volatility in dollar terms, independent of the price level.

In [12]:
df['ATR_14'] = ta.atr(data['High']['MSFT'], data['Low']['MSFT'], data['Close']['MSFT'], length=14)
df['ATR_14'].tail()

Date
2023-12-22    5.808482
2023-12-26    5.635695
2023-12-27    5.391499
2023-12-28    5.174600
2023-12-29    5.063981
Name: ATR_14, dtype: float64

### On-balance volume (OBV)
that uses trading volume to anticipate changes in stock prices. It signals potential price movements based on volume shifts, as volume typically precedes price action.
Measures buying and selling pressure. If price rises but OBV falls, it suggests the price rise lacks conviction and may be a bull trap (divergence).

In [14]:
df['OBV'] = ta.obv(df['Close'], df['Volume'])
df['OBV'].tail()

Date
2023-12-22    2.334676e+09
2023-12-26    2.347350e+09
2023-12-27    2.332444e+09
2023-12-28    2.346771e+09
2023-12-29    2.365502e+09
Name: OBV, dtype: float64

## TRADER BEHAVIOR (gemini wrote)

Indicator,Trader Behavior Captured <br>
SMA/EMA,"Trend Following: Identifies whether the consensus trend is up or down, driving traders to ""follow the herd."""
<br>MACD,"Momentum Crossover: Captures the rate of change in momentum, signaling when traders believe the acceleration/deceleration is strong enough to initiate a trade."
<br>RSI,"Overbought/Oversold Reversion: Identifies when buying/selling pressure is stretched to unsustainable extremes, prompting ""contrarian"" traders to anticipate a reversal."
<br>Bollinger Bands,"Volatility Expansion/Contraction: Captures periods of low volatility (calm before the storm) or extreme volatility (price touching the band), driving breakout or fade strategies."
<br>ATR,"Risk Sizing: Used by traders to size positions based on the asset's current ""natural"" price movement, ensuring stop losses are placed outside the typical noise."
<br>OBV,"Volume Confirmation: Measures whether large institutional money is validating the current price move, capturing underlying conviction and smart money flows."